# Actividad 4 | Métricas de calidad de resultados
---
- Alonso Pedrero Martínez   |   A01769076

## Selección de los datos
---

In [32]:
import findspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.types import StringType, DoubleType, FloatType
import findspark

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import pandas as pd
from os import path

In [2]:
findspark.init()
findspark.find()

'/Users/alonsopedreromartinez/Documents/GitHub/big_data/act 4/act_4_big_data/lib/python3.12/site-packages/pyspark'

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/08 22:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
PATH = "../files"
FILE = "amazon_electronics.csv"

In [5]:
class FileManager():
    @staticmethod
    def open_csv_file(input_path : str, file_name : str):
        """
        This method opens a csv file with pyspark
        """
        csv_df = spark.read.csv(
            path.join(input_path, file_name),
            header=True,
            inferSchema=True,
            multiLine=True,
            escape="\"",
            quote="\""
        )

        csv_df.show(truncate=20)

        return csv_df

In [6]:
df_reviews = FileManager.open_csv_file(PATH, FILE)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|sentiment|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|        1|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub 

In [7]:
# Previously defined relevant columns for the activity.
RELEVANT_COLUMNS_FOR_CHARACTERIZATION = [
  "star_rating",
  "helpful_votes",
  "total_votes",
  "vine",
  "verified_purchase",
  "review_date",
  "sentiment"
]

In [8]:
df_reviews_filtered = df_reviews.select(*RELEVANT_COLUMNS_FOR_CHARACTERIZATION)

### Generación de las particiones

Se implementó un procedimiento automático en PySpark que:

1. Filtra los registros de la base de datos que cumplen con cada combinación de valores.
2. Almacena cada subconjunto en un diccionario indexado por nombre de combinación (ej. "R5_VPY_VN" para `star_rating`=5, `verified_purchase`=Y, `vine`=N).
3. Imprime la cantidad de registros por partición para control y trazabilidad.

Las particiones con muy pocos registros pueden ser descartadas en etapas posteriores para evitar problemas en el análisis.

In [9]:
class PartitioningManager:

    @staticmethod
    def compute_probabilities(df, cols):
        """
        Computes and returns the probability of each combination of values in the specified columns.
        """
        total_count = df.count()
        return df.groupBy(cols).count() \
                 .withColumn("probability", F.round(F.col("count") / total_count, 6)) \
                 .orderBy("probability", ascending=False)

    @staticmethod
    def filter_partition(df, star_rating, verified_purchase, vine):
        """
        Filters the DataFrame by specific values for rating, verified purchase, and vine.
        """
        return df.filter(
            (F.col("star_rating") == star_rating) &
            (F.col("verified_purchase") == verified_purchase) &
            (F.col("vine") == vine)
        )

    @staticmethod
    def generate_all_partitions(df, min_probability=0.0001):
        """
        Generates partitions only for combinations whose joint probability is above min_probability.
        """

        prob_df = PartitioningManager.compute_probabilities(
            df, ["star_rating", "verified_purchase", "vine"]
        )

        filtered_combinations = prob_df.filter(
            F.col("probability") >= min_probability
        ).select("star_rating", "verified_purchase", "vine").collect()

        partitions = {}
        for row in filtered_combinations:
            rating = row["star_rating"]
            purchase = row["verified_purchase"]
            vine = row["vine"]

            key = f"R{rating}_VP{purchase}_V{vine}"
            filtered = PartitioningManager.filter_partition(df, rating, purchase, vine)
            partitions[key] = filtered
            print(f"Partition {key} created with {filtered.count()} records.")

        return partitions

    @staticmethod
    def stratified_sample_partitioned_data(partitions_dict, label_col="sentiment", fraction=0.3, min_rows=50):
        """
        Applies stratified sampling to each partition based on sentiment.
        """
        sampled_partitions = {}

        for key, df in partitions_dict.items():
            count = df.count()

            if count < min_rows:
                print(f"Skipping partition {key} — only {count} rows (<{min_rows})")
                continue

            sentiments = df.select(label_col).distinct().rdd.flatMap(lambda x: x).collect()
            fractions = {s: fraction for s in sentiments}

            sampled_df = df.sampleBy(label_col, fractions, seed=42)
            sampled_partitions[key] = sampled_df
            print(f"Sampled {sampled_df.count()} rows from partition {key} (original: {count})")

        return sampled_partitions

    @staticmethod
    def build_combined_sample(partitions_sampled_dict):
        """
        Unites all sampled partitions into a single DataFrame (M).
        This helps reduce computational load while maintaining diversity.
        """
        if not partitions_sampled_dict:
            raise ValueError("No partitions provided for sample combination.")

        combined_df = None
        for key, df in partitions_sampled_dict.items():
            if combined_df is None:
                combined_df = df
            else:
                combined_df = combined_df.union(df)
            print(f"Partition {key} added to the combined sample.")

        print(f"Total records in combined sample: {combined_df.count()}")
        return combined_df


In [10]:
partitions = PartitioningManager.generate_all_partitions(df_reviews, min_probability=0.00001)

Partition R5_VPY_VN created with 3679909 records.


Partition R4_VPY_VN created with 1019728 records.


Partition R1_VPY_VN created with 603371 records.


Partition R3_VPY_VN created with 443364 records.


Partition R5_VPN_VN created with 410073 records.


...

Partition R4_VPN_VY created with 13240 records.


Partition R3_VPN_VY created with 4886 records.


Partition R2_VPN_VY created with 1634 records.


Partition R1_VPN_VY created with 705 records.


Partition R5_VPY_VY created with 101 records.


In [11]:
# Sample of one of the generated partitions.
partitions["R4_VPY_VN"].show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|sentiment|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+
|         US|   49329488|R1QF6RS1PDLU18|B00TR05L9Y|     778403103|Lenovo TAB2 A10 -...|              PC|          4|            1|          1|   N|                Y|                Good|I am not sure I d...| 2015-08-31|        1|
|         US|   43341796|R2NJ3WFUS4E5G6|B00YGJJQ6U|     986548413|Fintie iPad Ai

## Técnica de muestreo aplicada por partición

Una vez construidas las particiones, se aplicó una técnica de **muestreo estratificado** sobre cada subconjunto, usando la variable `sentiment` como variable de estratificación. Esto asegura que cada muestra mantenga la proporción original de clases de sentimiento en la partición.

Para evitar particiones con tamaños insuficientes, se definió un umbral mínimo (`min_rows`) que descarta automáticamente las particiones con muy pocos registros.

Además, se permitió configurar el porcentaje de muestreo (`fraction`) por clase de sentimiento. Esto ofrece flexibilidad para ajustar el tamaño del conjunto de entrenamiento o validación según necesidades posteriores.

#### Justificación del muestreo estratificado

* **Preservación del equilibrio de clases**: Al muestrear por clase de sentimiento se evitan sesgos por clases desbalanceadas.
* **Relevancia contextual**: Al aplicar el muestreo dentro de cada partición (y no sobre la base completa), se conserva la variabilidad contextual de las reseñas.
* **Evita el submuestreo accidental**: Las particiones muy pequeñas son descartadas de forma controlada, garantizando que el conjunto final tenga representatividad suficiente.

In [12]:
sampled_partitions = PartitioningManager.stratified_sample_partitioned_data(partitions, fraction=0.05, min_rows=100)

Sampled 184082 rows from partition R5_VPY_VN (original: 3679909)


Sampled 51148 rows from partition R4_VPY_VN (original: 1019728)


Sampled 30183 rows from partition R1_VPY_VN (original: 603371)


Sampled 22223 rows from partition R3_VPY_VN (original: 443364)


Sampled 20507 rows from partition R5_VPN_VN (original: 410073)


...

Sampled 724 rows from partition R4_VPN_VY (original: 13240)


Sampled 266 rows from partition R3_VPN_VY (original: 4886)


Sampled 93 rows from partition R2_VPN_VY (original: 1634)


Sampled 35 rows from partition R1_VPN_VY (original: 705)


Sampled 3 rows from partition R5_VPY_VY (original: 101)


In [13]:
df_sample_M = PartitioningManager.build_combined_sample(sampled_partitions)

Partition R5_VPY_VN added to the combined sample.
Partition R4_VPY_VN added to the combined sample.
Partition R1_VPY_VN added to the combined sample.
Partition R3_VPY_VN added to the combined sample.
Partition R5_VPN_VN added to the combined sample.
Partition R2_VPY_VN added to the combined sample.
Partition R1_VPN_VN added to the combined sample.
Partition R4_VPN_VN added to the combined sample.
Partition R3_VPN_VN added to the combined sample.
Partition R2_VPN_VN added to the combined sample.
Partition R5_VPN_VY added to the combined sample.
Partition R4_VPN_VY added to the combined sample.
Partition R3_VPN_VY added to the combined sample.
Partition R2_VPN_VY added to the combined sample.
Partition R1_VPN_VY added to the combined sample.
Partition R5_VPY_VY added to the combined sample.


Total records in combined sample: 345829


## Preparación del conjunto de entrenamiento y prueba
--- 

In [14]:
class StatisticalAnalysisHelper():
    @staticmethod
    def dataset_dimensions(df_input):
        print("columns in the dataset:", len(df_input.columns))
        print("rows in the dataset:", df_input.count())

    @staticmethod
    def schema_information(df_input):
        """
        This method shows the current schema of the data.
        """
        df_input.printSchema()

    @staticmethod
    def descriptive_statistics(df_input):
        """
        This method shows the descriptive statistics of the data.
        """
        df_input.summary().show(truncate=False)

    @staticmethod
    def missing_values_table(df_input):
        """
        Displays a table with the count of missing values per column.
        """
        missing_exprs = []
        
        for c in df_input.schema.fields:
            field_name = c.name
            field_type = c.dataType
            
            if isinstance(field_type, (DoubleType, FloatType)):
                missing_exprs.append(
                    count(when(col(field_name).isNull() | isnan(col(field_name)), field_name)).alias(field_name)
                )
            elif isinstance(field_type, StringType):
                missing_exprs.append(
                    count(when(col(field_name).isNull() | (col(field_name) == ""), field_name)).alias(field_name)
                )
            else:
                missing_exprs.append(
                    count(when(col(field_name).isNull(), field_name)).alias(field_name)
                )

        df_missing_values = df_input.select(missing_exprs)

        return df_missing_values

In [15]:
StatisticalAnalysisHelper.dataset_dimensions(df_sample_M)

columns in the dataset: 16


rows in the dataset: 345829


In [16]:
StatisticalAnalysisHelper.schema_information(df_sample_M)

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- sentiment: integer (nullable = true)



Este esquema describe la estructura de un conjunto de datos de reseñas de productos, ya preparado para análisis en PySpark. Todas las columnas están correctamente tipadas para facilitar tareas de preprocesamiento, modelado y visualización, por lo que no requieren cambios de tipo adicionales. A continuación se describe cada variable:

- marketplace (string): Código del país o región del mercado (por ejemplo, "US", "MX").
- customer_id (integer): Identificador único del cliente que realizó la reseña.
- review_id (string): Identificador único de la reseña.
- product_id (string): Identificador único del producto reseñado.
- product_parent (integer): ID grupal de productos relacionados (puede representar variantes).
- product_title (string): Nombre o título del producto.
- product_category (string): Categoría del producto.
- star_rating (integer): Calificación en estrellas otorgada al producto (usualmente de 1 a 5).
- helpful_votes (integer): Número de votos que consideraron útil la reseña.
- total_votes (integer): Número total de votos recibidos por la reseña.
- vine (string): Indica si la reseña es parte del programa Vine ("Y" o "N").
- verified_purchase (string): Indica si la reseña proviene de una compra verificada ("Y" o "N").
- review_headline (string): Título de la reseña.
- review_body (string): Cuerpo completo del texto de la reseña.
- review_date (date): Fecha en que se publicó la reseña.
- sentiment (integer): Etiqueta de sentimiento preprocesada, usada como variable objetivo para modelos supervisados (por ejemplo, 1 = positivo, 0 = negativo).


Este esquema permite realizar tanto análisis exploratorio como modelado predictivo con modelos de machine learning supervisados y no supervisados en PySpark sin necesidad de transformación adicional de tipos de datos.

In [17]:
StatisticalAnalysisHelper.descriptive_statistics(df_sample_M)

25/06/08 22:32:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------+--------------------+--------------+-------------------+--------------------+----------------------------------------------------------------------------------------------+----------------+------------------+------------------+------------------+------+-----------------+----------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|summary|marketplace|customer_id         |review_id     |product_id         |product_parent      |product_title                                  

In [18]:
missing_values = StatisticalAnalysisHelper.missing_values_table(df_sample_M)
missing_values.show(truncate=False)

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+---------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|sentiment|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+---------+
|0          |0          |0        |0         |0             |0            |0               |0          |0            |0          |0   |0                |0              |0          |0          |0        |
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+---

El contenido mostrado indica que el dataset no contiene valores nulos, vacíos ni faltantes en ninguna de sus columnas, lo cual es ideal para su uso en modelos de machine learning. Todas las columnas tienen registros válidos, incluso aquellas de tipo texto, entero o fecha. La fila con valores "0" representa probablemente un ejemplo simbólico o una validación inicial de integridad, no un error.

Dado que no hay datos vacíos ni inconsistentes, no se requieren procesos adicionales de limpieza, imputación ni eliminación de filas o columnas, lo que permite avanzar directamente al análisis exploratorio, transformación de características o entrenamiento de modelos con confianza en la calidad del conjunto de datos.

## Preparación del conjunto de entrenamiento y prueba
--- 

In [19]:
class TrainTestManager:
  @staticmethod
  def stratified_train_test_split(df, label_col, train_ratio : int, seed : int):
    """
    Performs a stratified split of the DataFrame based on the label column.
    Returns (train_df, test_df).
    """
    label_values = df.select(label_col).distinct().rdd.flatMap(lambda x: x).collect()

    train_fractions = {label: train_ratio for label in label_values}

    train_df = df.sampleBy(label_col, train_fractions, seed=seed)

    train_ids = train_df.select(F.monotonically_increasing_id().alias("id"))
    df_with_id = df.withColumn("id", F.monotonically_increasing_id())

    test_df = df_with_id.join(train_ids, on="id", how="left_anti").drop("id")
    train_df = train_df.drop("id") if "id" in train_df.columns else train_df

    print(f"Train set: {train_df.count()} rows")
    print(f"Test set: {test_df.count()} rows")

    return train_df, test_df

## COnstrucción de conjunto de entrenamiento y prueba
---

Se realizó una división estratificada del DataFrame df_sample_M utilizando la clase TrainTestManager. Esto significa que los datos fueron separados en entrenamiento y prueba manteniendo la proporción original de las clases presentes en la columna "sentiment", que es la variable objetivo.


Recopila todos los valores únicos de la clase (label_col = "sentiment").
Aplica un muestreo estratificado con una fracción de 80% para entrenamiento (train_ratio = 0.8) y el 20% restante para prueba.
Usa una semilla fija (seed = 42) para garantizar reproducibilidad.
Se asegura de que los datos del conjunto de prueba no se solapen con los del conjunto de entrenamiento.

Resultado de la división:

Training set: 276,731 muestras (80%)
Test set: 69,098 muestras (20%)


Este enfoque mejora la generalización del modelo, especialmente cuando se trabaja con clases desbalanceadas, ya que cada clase está representada proporcionalmente en ambos conjuntos.

In [20]:
train_df, test_df = TrainTestManager.stratified_train_test_split(df_sample_M, "sentiment", 0.8, 42)

Train set: 276731 rows


Test set: 69098 rows


## Selección de métricas de calidad
---
Accuracy:
>Accuracy mide la proporción de predicciones correctas que hace el modelo sobre el total de casos. Es decir, cuántas veces el modelo acierta, sin distinguir entre clases. Es útil cuando las clases están balanceadas y los errores tienen el mismo peso. Sin embargo, puede ser engañosa si hay desbalance de clases, ya que un modelo puede tener alta exactitud simplemente prediciendo siempre la clase mayoritaria.

Precision
>La precisión (precision) es una métrica que indica qué proporción de las predicciones positivas realizadas por el modelo fueron correctas. Es especialmente importante en contextos donde los falsos positivos tienen un alto costo, como en sistemas de detección de fraude, diagnósticos médicos o filtrado de spam. Una alta precisión significa que el modelo es confiable al identificar positivos, ya que comete pocos errores al etiquetar como tal. Su importancia radica en que permite evaluar la exactitud del modelo cuando afirma que una instancia pertenece a una clase positiva, ayudando a evitar consecuencias negativas derivadas de decisiones incorrectas.

f1_score
>F1-score es la media armónica entre la precisión (precision) y la exhaustividad (recall). Es una métrica clave cuando hay un desbalance entre clases o cuando los falsos positivos y falsos negativos tienen consecuencias importantes. Sirve para tener una visión más equilibrada del desempeño del modelo que usando solo accuracy, ya que castiga los extremos donde una métrica es alta y la otra baja.

precision
>Precision indica qué proporción de las predicciones positivas realmente lo son. Es esencial cuando el costo de un falso positivo es alto. Por ejemplo, si un modelo predice que un producto es defectuoso y no lo es, puede generar gastos innecesarios. Por eso, esta métrica evalúa qué tan "confiables" son los aciertos positivos del modelo.

recall
>Recall (también llamado sensibilidad o exhaustividad) mide la capacidad del modelo para identificar todos los casos positivos reales. Es vital cuando los falsos negativos son especialmente graves, como en el diagnóstico médico o detección de fraudes. Un alto recall asegura que el modelo no está dejando pasar demasiados positivos verdaderos sin detectar.

ROC AUC
>La ROC AUC (Receiver Operating Characteristic - Area Under the Curve) es una métrica que evalúa la capacidad de un modelo para distinguir entre clases positivas y negativas. La curva ROC compara la tasa de verdaderos positivos (recall) frente a la tasa de falsos positivos a diferentes umbrales de decisión, y el área bajo esta curva (AUC) resume el rendimiento del modelo en un solo valor entre 0 y 1. Un AUC de 1.0 indica un modelo perfecto, mientras que 0.5 sugiere un modelo sin capacidad predictiva (equivalente a adivinar al azar). Su importancia radica en que es independiente del umbral y útil especialmente cuando hay clases desbalanceadas, ya que proporciona una visión global del comportamiento del modelo más allá de una única métrica como precisión o recall.

## Construcción de modelo de aprendizaje
--- 
La clase ManualLogisticRegressionManager se encarga de gestionar todo el proceso necesario para entrenar, preprocesar y evaluar un modelo de regresión logística binaria utilizando PySpark. Primero, permite preparar los datos mediante un pipeline que ensambla las variables predictoras y las escala para que tengan una magnitud comparable. Luego, entrena el modelo aplicando validación cruzada con una búsqueda de hiperparámetros (regParam y elasticNetParam) para encontrar la mejor configuración posible, evaluando el rendimiento con el área bajo la curva ROC (AUC). Finalmente, evalúa el modelo generando métricas clave como la precisión, recall, F1-score, accuracy y el propio AUC, lo que permite tener una visión completa del desempeño del modelo en tareas de clasificación binaria. Esta clase automatiza y organiza eficientemente cada una de las etapas del flujo de trabajo del modelo.

In [33]:
class ManualLogisticRegressionManager:
    @staticmethod
    def evaluate_binary_classification(predictions, label_col: str = "label", prediction_col: str = "prediction", probability_col: str = "probability"):
        """
        Evalúa un modelo de clasificación binaria en base a varias métricas estándar.
        """
        accuracy = predictions.filter(F.col(label_col) == F.col(prediction_col)).count() / predictions.count()
        
        tp = predictions.filter((F.col(prediction_col) == 1) & (F.col(label_col) == 1)).count()
        fp = predictions.filter((F.col(prediction_col) == 1) & (F.col(label_col) == 0)).count()
        fn = predictions.filter((F.col(prediction_col) == 0) & (F.col(label_col) == 1)).count()
        
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0

        evaluator = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol=probability_col, metricName="areaUnderROC")
        auc = evaluator.evaluate(predictions)

        results_df = pd.DataFrame({
            "accuracy": [accuracy],
            "precision": [precision],
            "recall": [recall],
            "f1_score": [f1],
            "auc": [auc]
        })

        return results_df
    
    @staticmethod
    def preprocess_data(df_train, df_test, feature_cols, label_col="sentiment"):
        """
        Arma un pipeline para ensamblar y escalar características.
        """
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vec")
        scaler = StandardScaler(inputCol="features_vec", outputCol="features", withStd=True, withMean=False)
        pipeline = Pipeline(stages=[assembler, scaler])

        pipeline_model = pipeline.fit(df_train)
        train_preprocessed = pipeline_model.transform(df_train)
        test_preprocessed = pipeline_model.transform(df_test)

        return train_preprocessed, test_preprocessed

    @staticmethod
    def train_logistic_regression(train_df, test_df, label_col="sentiment"):
        """
        Trains a logistic regression model on the training set and evaluates it on the test set.
        Returns the trained model and evaluation metrics.
        """
        lr = LogisticRegression(labelCol=label_col, featuresCol="features", maxIter=10)

        paramGrid = (ParamGridBuilder()
            .addGrid(lr.regParam, [0.1])
            .addGrid(lr.elasticNetParam, [0.0])
            .build())

        evaluator = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

        cv = CrossValidator(estimator=lr,
                            estimatorParamMaps=paramGrid,
                            evaluator=evaluator,
                            numFolds=2,
                            parallelism=2)

        cv_model = cv.fit(train_df)
        best_model = cv_model.bestModel

        predictions = best_model.transform(test_df)

        print(f"BEst model: RegParam = {best_model._java_obj.getRegParam()}, ElasticNet = {best_model._java_obj.getElasticNetParam()}")
        
        return predictions, best_model

In [34]:
features = ["star_rating", "helpful_votes", "total_votes"]

train_pp, test_pp = ManualLogisticRegressionManager.preprocess_data(
  train_df.sample(withReplacement=False, fraction=0.1, seed=42),
  test_df.sample(withReplacement=False, fraction=0.1, seed=42),
  feature_cols=features
)

## Análisis de resultados
--- 


In [35]:
predictions, best_model = ManualLogisticRegressionManager.train_logistic_regression(train_pp, test_pp)

BEst model: RegParam = 0.1, ElasticNet = 0.0


In [36]:
metrics = ManualLogisticRegressionManager.evaluate_binary_classification(predictions, label_col="sentiment")

In [37]:
display(metrics)

,accuracy,precision,recall,f1_score,auc
0,0.933079,0.910404,0.999809,0.953014,0.999809


El modelo de regresión logística binaria obtuvo resultados excepcionales según las métricas evaluadas, lo que indica un rendimiento altamente confiable y preciso. La exactitud (accuracy) alcanzó un 93.31%, lo que significa que el modelo clasificó correctamente más del 93% de los casos en general, una cifra muy alta que refleja su consistencia global.

La precisión fue de 91.04%, lo que implica que, de todas las instancias que el modelo predijo como positivas, más del 91% realmente lo eran. Esto es especialmente importante en contextos donde los falsos positivos tienen un alto costo, ya que ayuda a reducir errores en decisiones automatizadas.

Por otro lado, el recall fue prácticamente perfecto, con un 99.98%, lo que indica que el modelo logró identificar casi todos los casos positivos reales. Esta métrica es crucial en aplicaciones donde los falsos negativos son críticos, como en la detección de fraudes, diagnósticos médicos o control de calidad.

El F1 Score, que combina precisión y recall en una única métrica armónica, alcanzó un 95.30%, lo que muestra un excelente balance entre ambas capacidades. Esto refuerza que el modelo no solo es preciso, sino también exhaustivo al identificar correctamente las clases.

Finalmente, el AUC-ROC fue de 0.9998, una métrica que evalúa la capacidad del modelo para distinguir entre las clases. Un valor tan cercano a 1 demuestra que el modelo tiene un poder discriminativo casi perfecto, es decir, que puede separar muy bien los positivos de los negativos en diferentes umbrales de decisión.

Tiene un alto desempeño y se valida la solidez del modelo para ser implementado en entornos de producción donde se requiere alta confiabilidad y mínimo margen de error